In [1]:
import requests
import pandas as pd
import os
import re
import multiprocessing
from tqdm import tqdm
from bs4 import BeautifulSoup as bs

import warnings
warnings.filterwarnings('ignore')

In [2]:
data = pd.read_csv("page_product.csv")

no_review_ids = []
userid_set = set()

In [3]:
data

,page_id,product_id
0,133556,32125
1,133556,33050
2,133556,75758
3,133556,101921
4,133556,110328
...,...,...
345266,318,2258
345267,318,2259
345268,318,2261
345269,318,2262


In [4]:
def review_crawl(result, no_review_ids, product_number):
    url = f"https://ohou.se/production_reviews.json?production_id={product_number}&page=1&order=best&photo_review_only="
    response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"})
    if response.status_code != 200:
        return
    review = response.json()
    counts = review['total_count']
    if counts == 0:
        no_review_ids.append(product_number)
        return

    for i in range(1, counts//1000+2):
        url = f"https://ohou.se/production_reviews.json?production_id={product_number}&page={i}&order=best&photo_review_only=&per=1000"
        response = requests.get(url, headers={"User-Agent": "Mozilla/5.0"}, verify=False)
        review = response.json()['reviews']

        for j in review:
            userid_set.add(j['writer_id'])
            data = [product_number, j['writer_id'], j['created_at'], j['review']['star_avg'], j['review']['star_durability'], j['review']['star_design'], j['review']['star_cost'],  j['review']['star_delivery'],  j['review']['comment'], j['production_information']['explain']]
            # data = {'itemid': product_number, 'userid': j['writer_id'], 'timestamp': j['created_at'], \
            # 'star_avg': j['review']['star_avg'], 'star_durability': j['review']['star_durability'], \
            # 'star_design': j['review']['star_design'], 'star_cost': j['review']['star_cost'], \
            # 'star_delivery': j['review']['star_delivery'], 'comment': j['review']['comment'], \
            # 'option': j['production_information']['explain']}
            # dataaa = pd.DataFrame(data, index=[0])
            # review_df = pd.concat([review_df, dataaa])
            result.append(data)


In [5]:
data['product_id'].nunique()

131408

In [6]:
import numpy as np
np.cumsum([data['product_id'].nunique()/4] * 4)

array([ 32852.,  65704.,  98556., 131408.])

In [7]:
# base = 'https://ohou.se/productions'
# header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

if __name__ == "__main__":
    manager = multiprocessing.Manager()
    result = manager.list()
    no_review_ids = manager.list()
    pool=multiprocessing.Pool(processes=4)
    a=[]
    for item_id in tqdm(data['product_id'].unique()[0:32852]):
        p = multiprocessing.Process(target=review_crawl, args=(result, no_review_ids, item_id))
        a.append(p)
        p.start()

    for proc in a:
        proc.join()


  3%|▎         | 1025/32852 [00:19<21:13, 24.99it/s]Process Process-88:
Traceback (most recent call last):
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/connectionpool.py", line 703, in urlopen
    httplib_response = self._make_request(
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/connectionpool.py", line 386, in _make_request
    self._validate_conn(conn)
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/connectionpool.py", line 1042, in _validate_conn
    conn.connect()
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/connection.py", line 414, in connect
    self.sock = ssl_wrap_socket(
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/util/ssl_.py", line 449, in ssl_wrap_socket
    ssl_sock = _ssl_wrap_socket_impl(
  File "/opt/conda/envs/movierec/lib/python3.8/site-packages/urllib3/util/ssl_.py", line 493, in _ssl_wrap_socket_impl
    return ssl_context.wrap_socket(sock, server_

In [8]:
len(list(no_review_ids))

7498

In [9]:
df = pd.DataFrame(list(result), columns=['itemid', 'userid', 'timestamp', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'comment', 'option'])

In [34]:
df['itemid'].nunique()

25327

In [12]:
32852 - (25327 + 7498)

27

In [24]:
rest_product_ids = list(set(data['product_id'].unique()[0:32852]) - (set(list(no_review_ids)).union( set(df['itemid']))))

In [25]:
# base = 'https://ohou.se/productions'
# header = {"user-agent":"Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36"}

if __name__ == "__main__":
    manager = multiprocessing.Manager()
    result = manager.list()
    no_review_ids = manager.list()
    pool=multiprocessing.Pool(processes=4)
    a=[]
    for item_id in tqdm(rest_product_ids):
        p = multiprocessing.Process(target=review_crawl, args=(result, no_review_ids, item_id))
        a.append(p)
        p.start()

    for proc in a:
        proc.join()


100%|██████████| 27/27 [00:00<00:00, 35.39it/s]


In [36]:
list(no_review_ids)

[]

In [29]:
df_rest = pd.DataFrame(list(result), columns=['itemid', 'userid', 'timestamp', 'star_avg', 'star_durability', 'star_design', 'star_cost', 'star_delivery', 'comment', 'option'])

In [30]:
df_all = pd.concat([df, df_rest], axis=0)

In [42]:
df_all.itemid.nunique() + 7498

32851

In [49]:
set(rest_product_ids) - set(df_all.itemid)

{177264}

In [53]:
df_all

,itemid,userid,timestamp,star_avg,star_durability,star_design,star_cost,star_delivery,comment,option
0,118895,7001668,2020.12.22,5.0,0,0,0,0,편안하고 깔끔하구요\n살짝 좌식 시 미끄러운감은 있지만\n사용에 문제없어요,
1,118895,5813881,2020.04.26,3.0,0,0,0,0,화이트는 좀 관리가 힘든거 같아요... 배송좀신경써주셨으면...,
2,118895,6447991,2020.03.05,5.0,0,0,0,0,아직 식탁이 안와서 의자사진만 있지만 튼튼하고 지정일 맞춰서 보내주셨어요~!,
3,118895,710254,2020.03.15,5.0,0,0,0,0,앉아있을때 너무나 편안하고 오래 앉아도 편합니다:). 예쁘면서도 실용적이라면 말다...,
4,321021,367873,2020.08.28,5.0,5,5,5,5,첫 마디~~ 너무 예뻐요 깜직한 디자인에 거름망도 아주 정교하게 촘촘해서 차 마실...,단품
...,...,...,...,...,...,...,...,...,...,...
5062,310459,2821036,2020.04.12,4.0,2,4,5,5,재질이 얇은거 빼고 보슬보슬하고 괜찮아요.,색상: 01_크림 / 사이즈: 150x200
5063,310459,5416726,2020.04.07,5.0,5,5,5,5,가격도 저렴하고 배송도 빠르고 무엇보다 미끌거리지 않아서 좋네요 ! 러그 바닥이 힘...,색상: 01_크림 / 사이즈: 100x150
5064,310459,3294881,2020.04.03,4.5,3,5,5,5,배송 진짜 빨리오고 부드러워요! 색깔이 조금 누런?느낌이 있는데 나쁘진 않아요,색상: 01_크림 / 사이즈: 150x200
5065,310459,7592634,2020.04.02,5.0,5,5,5,5,디자인 가격 모두 맘에 들어요! 제가 좀 작은 사이즈를 산거 빼곤... ㅎ 그런데 ...,색상: 02_그레이 / 사이즈: 100x150


In [62]:
df_all_drop_duplicates = df_all.drop_duplicates()

In [63]:
df_all_drop_duplicates.to_csv("review_0_32852.csv", index=False)

In [65]:
df_all_drop_duplicates.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 8948311 entries, 0 to 5066
Data columns (total 10 columns):
 #   Column           Dtype  
---  ------           -----  
 0   itemid           int64  
 1   userid           int64  
 2   timestamp        object 
 3   star_avg         float64
 4   star_durability  int64  
 5   star_design      int64  
 6   star_cost        int64  
 7   star_delivery    int64  
 8   comment          object 
 9   option           object 
dtypes: float64(1), int64(6), object(3)
memory usage: 751.0+ MB
